<a href="https://colab.research.google.com/github/squeeko/TensorflowProjects/blob/in_progress/TF_PreProcess_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>